In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/cms/processed/metadata')
db_path=os.path.join(root_dir,'db/cms.db')

In [3]:
#define input tables and metadata
tbl_names_lst=['s1_ben_sum_2008','s1_ben_sum_2009','s1_ben_sum_2010','s1_carrier_1a','s1_carrier_1b','s1_inpatient','s1_outpatient','s1_prescrp']

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    'LESS_GRP_VARS': False, # enforce bias in random queries toward smaller number of groupby vars. Default is no bias (i.e. uniform sampling)
    'LESS_CMP_VARS':False, # enforce bias in random queries toward small number of  comparison terms. Default is no bias (i.e. uniform sampling)
    'JOIN_CNDTN':{'WHERE':0.5, 'AND':0.5} #Use WHERE or AND with JOIN CLAUSE
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

smk2


In [5]:
# Generate Random queries 


# rnd_query=self.make_single_agg_query()

syn_tbl_name_lst=['s2_ben_sum_2008','s2_ben_sum_2009','s2_ben_sum_2010','s2_carrier_2a','s2_carrier_2b','s2_inpatient','s2_outpatient','s2_prescrp']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst)


['s1_carrier_1b', 's1_outpatient', 's1_prescrp', 's1_carrier_1a', 's1_inpatient'] 5


In [6]:
rnd_query['query_desc']

{'type': 'twin_agg',
 'agg_fntn': 'None',
 'grpby_vars': ['LINE_PRCSG_IND_CD_11',
  'LINE_PRCSG_IND_CD_1',
  'SP_STATE_CODE',
  'ICD9_DGNS_CD_2',
  'SEGMENT',
  'HCPCS_CD_8',
  'DESYNPUF_ID',
  'HCPCS_CD_25',
  'HCPCS_CD_41',
  'CLM_DRG_CD',
  'BENE_ESRD_IND',
  'LINE_ICD9_DGNS_CD_9',
  'ICD9_DGNS_CD_1',
  'DESYNPUF_ID',
  'BENRES_OP',
  'LINE_PRCSG_IND_CD_10',
  'ICD9_DGNS_CD_7',
  'SP_ISCHMCHT',
  'CLM_FROM_DT',
  'DESYNPUF_ID',
  'BENE_RACE_CD',
  'ICD9_DGNS_CD_6',
  'ICD9_DGNS_CD_5',
  'DESYNPUF_ID',
  'MEDREIMB_OP',
  'HCPCS_CD_1',
  'LINE_ICD9_DGNS_CD_6',
  'HCPCS_CD_10',
  'HCPCS_CD_27',
  'HCPCS_CD_13',
  'ICD9_DGNS_CD_5',
  'HCPCS_CD_33',
  'HCPCS_CD_38',
  'HCPCS_CD_4',
  'HCPCS_CD_11',
  'LINE_ICD9_DGNS_CD_11',
  'HCPCS_CD_8',
  'HCPCS_CD_22',
  'LINE_ICD9_DGNS_CD_13',
  'HCPCS_CD_3',
  'DESYNPUF_ID',
  'HCPCS_CD_37',
  'ICD9_PRCDR_CD_5',
  'HCPCS_CD_9',
  'SEGMENT',
  'LINE_PRCSG_IND_CD_11',
  'TAX_NUM_1',
  'LINE_ICD9_DGNS_CD_4',
  'SP_CNCR',
  'ICD9_DGNS_CD_3',
  'MEDREIM